In [108]:
import numpy as np
import matplotlib.pyplot as plt
import copy

### 3 L-layer NN

hidden lyaer가 없는건, 판별선이 선형으로만 이뤄져 한계가 있었고
hidden lyaer가 하나 이상 있는것 또한 복잡한 패턴을 인식하는데에 있어서 쉽지 않았음.
따라서 이 hidden layer를 2개 혹은 n개로 늘릴 것임
이때 당연히 hidden layer units은 하이퍼파라미터로 설정하게 될 것. 

w1 = ~~ W2 = ~ 했던 식보다 좀 더 현명한 방법을 쓰게 될 것임

In [71]:
# 1. 전에 n_x ,n_h, n_y 를 정했다면 이번에는 레이어가 몇개가 될지, 
# 그리고 그 hidden unit이 어떻게 될지 결정해야할 것

def initialize_parameters_deep(layer_dims):
    '''
    layer_dims = [n_x, n_h1, n_h2, ...,n_y]
    지난번에 배운대로 대칭+기울기소실 문제를 해결하기 위해서 w는 
    randn * 0.01을, b는 0으로 초기화
    '''
    
    # W1 W2 b2 등을 저장할 params라는 dictionary
    params = {} 

    # hidden layer가 1개인 상황이었다면
    # layer_dims = {n_x, n_h, n_y}
    # params에는 {w1 b1과 w2 b2가 필요할 것}
    # 따라서 len (layer_dims) - 1번만큼 실행되어야 함.
    L = len(layer_dims)
    for l in range(1, L): #{W1 : ...]} 과 같이 저장되게 됨
        #w의 차원 수는 (after_dim, before_dim)과 같은 식으로 설정
        params['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        params['b' + str(l)] = np.zeros((layer_dims[l], 1))

        assert(params['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(params['b' + str(l)].shape == (layer_dims[l], 1))

    return params



In [72]:
initialize_parameters_deep([3,4,5])

{'W1': array([[-0.00715987, -0.01001779, -0.01792443],
        [ 0.00630844, -0.00069599,  0.00053803],
        [-0.00059532, -0.00707596, -0.00427427],
        [ 0.00366613,  0.00270344,  0.0031276 ]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.]]),
 'W2': array([[-0.00307764, -0.01478633, -0.00290341, -0.00375866],
        [ 0.01017189,  0.01955388, -0.00550092,  0.00886382],
        [-0.01233847,  0.00175461,  0.01416491, -0.00815376],
        [ 0.00112716,  0.00583276, -0.01887958,  0.00100959],
        [ 0.00286092, -0.002107  , -0.01719883,  0.00756517]]),
 'b2': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]])}

In [73]:
def relu(Z):
    '''
    np.maximum
    a = (1,3,5)
    b = (6,4,2)
    np.maximum-> (6,4,5) 그 자리에서 높은 값 채워 넣음
    ReLU의 의도와 알맞는 친구다 
    '''
    return np.maximum(Z, 0)

In [74]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

In [75]:
def forward(A_prev, W, b, activation = "relu"):
    '''
    returns: A[l] , cache
    cache = (linear_cache , activation_cache)
    linear_cache = A[l-1], W[l], B[l]
    activation_cache = Z[l]
    '''

    Z = W @ A_prev + b
    linear_cache = (A_prev, W, b)

    if activation == "relu":
        A = relu(Z)

    elif activation == "sigmoid":
        A = sigmoid(Z)
    
    activation_cache = Z
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [76]:
A_prev = np.random.randn(4,1)
W = np.random.randn(5,4) * 0.01
b = np.zeros((5,1))
A, _ = forward(A_prev, W, b)
display(A)
del A_prev, W ,b, A, _

array([[0.01474844],
       [0.00284574],
       [0.00241597],
       [0.        ],
       [0.0151312 ]])

In [77]:
'''
L번 forward 하기
'''
def L_model_forward(X, params):
    '''
    X와 파라미터를 넣어주면, L번 순방향 전파를 해서 Y_hat 값을 출력
    return: y_hat, caches
    caches = linear_caches, activation_caches
    linear_caches:      (A[l-1], W[l], b[l])들
    activation_caches : (Z[l])들
    '''
    caches = []
    A = X
    
    # params = {W1: ...  , b1... , ..., WL: ..., bL: ... } 이기 때문에 
    # params의 len 은 2L임 (L은 레이어의 갯수)
    # 나누기는 값을 float로 반환하기 때문에 정수를 반환하는 // 몫을 사용
    # L = int(len(params) / 2)
    L = len(params) //2 

    for l in range(1, L):
        A_prev = A
        A, cache = forward(A_prev,
                           params[f'W{str(l)}'],
                           params[f'b{str(l)}'],
                           activation = "relu",
                           )
        caches.append(cache)
    # L-1번째까지 forward가 작동한 상황
    y_hat, cache = forward(A ,
                           params["WL"],
                           params["bL"],
                           activation = "sigmoid"
                           )
    caches.append(cache)
    
    return y_hat, caches

In [78]:
#손실 계산하기
def compute_cost(y_hat, Y):
    '''
    m개의 데이터셋에서 도출해낸 cost
    '''
    m = Y.shape[1]
    cost = np.sum(Y* np.log(y_hat) + (1-Y) * np.log(1-y_hat)) / -m
    cost = np.squeeze(cost)
    return cost

In [79]:
'''
forward  (A[l-1]) -- W,b ------> (Z[l]) -- activation --> A[l]  
backward (A[l-1]) <-- W,b ------ (Z[l]) -- activation <-- A[l],Y  
grad:  dA[l-1]<-- DW[l],Db[l] <-- DZ[l] <--------------- A[l], Y
과 같은 순서를 생각해볼 수 있음
backward는 dA[l-1]<-- DW[l],Db[l] <-- DZ[l] 의 과정을 하는 것
'''
def backward(dZ, cache):
    '''
    forward처럼, 기본적인 backward 정의
    forward가  Z = W@A_prev + B 에서
    input : W, A_prev, b   output : Z 라면 
    backward는 
    input : Z              output : W, A_prev, b 
    이지 않을까
    미리 계산된 식을 사용한다
    '''
    A_prev, W,b = cache # linear cache겠죵?

    #A_prev에서는 데이터의 수 m만큼 열이 있겠죵? 
    m = A_prev.shape[1]
    
    dW = (dZ @ A_prev.T)/m
    db = np.sum(dZ, axis= 1, keepdims= True)/m
    dA_prev = W.T@dZ

    return dA_prev, dW, db



In [80]:
def relu_backward(dA, cache):
    '''
    cache = Z
    '''
    Z = cache
    dZ = np.array(dA, copy = True)
    dZ[Z < 0] = 0
    return dZ

In [81]:
def sigmoid_backward(dA, cache):
    '''
    cache = Z
    '''
    Z = cache
    s = 1/(1+np.exp(-z))
    dZ = dA * s * (1-s)
    return dZ

In [82]:
'''
forward  (A[l-1]) -- W,b ------> (Z[l]) -- activation --> A[l]  
backward (A[l-1]) <-- W,b ------ (Z[l]) -- activation <-- A[l],Y  
grad:  dA[l-1]<-- DW[l],Db[l] <-- DZ[l] <--------------- A[l], Y
과 같은 순서를 생각해볼 수 있음
backward는 dA[l-1]<-- DW[l],Db[l] <-- DZ[l] 의 과정을 했다면
backward_with_activation은  DZ[l] <--------------- A[l], Y 의 과정을 하는 것
'''
def backward_with_activation(dA, cache, activation):
    '''
    cache = Z
    dZ = dA * ACTIVATION(Z)
    return : dA[l-1], dW[l], dB[l]
    근데 dA는 뭐 하늘에서 떨어지나요.. 그러게요
    최초의 dAL은 손실함수에 의해 계산되고 재귀적으로 dA[L-1]부터 dA[l]로 적용된답니다!

    '''
    linear_cache, activation_cache = cache

    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, dB = backward(dZ, linear_cache)
    
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, dB = backward(dZ, linear_cache)

    return dA_prev, dW, dB

In [ ]:
x1 = np.array([1,2,3,4])
x2 = np.array([4,3,2,1])
print(x1)
print(x2)
#print(np.divide(x1,x2))
print("""np.devide(x1,x2)는 x1을 x2로 나눕니다
그냥 나누기랑 똑같아요
근데 0으로 나누거나 하는 상황에서 처리 방식을 지정할 수 있음
result = np.divide([1,0,3] , [0,0,3] ,out = np.zeros_like(a), where=b!=0) -> b가 0이 아닐때만 나눗셈을 수행하겠다
""")
print(x1/x2)
del x1, x2

[1 2 3 4]
[4 3 2 1]
np.devide(x1,x2)는 x1을 x2로 나눕니다
위엣 식을 아래로 나눴다고 보면 되겠네요
0.25 = 1/4
0.66666667 = 2/3
1.5 = 3/2
4 = 4/1
[0.25       0.66666667 1.5        4.        ]


In [105]:
'''
forward  (A[l-1]) -- W,b ------> (Z[l]) -- activation --> A[l]  
backward (A[l-1]) <-- W,b ------ (Z[l]) -- activation <-- A[l],Y  
grad:  dA[l-1]<-- DW[l],Db[l] <-- DZ[l] <--------------- A[l], Y
그동한 계산한 caches들과 yhat,y만 있으면 backward할 수 있겠군!
최초의 dAL = -( (Y/Y_hat) - (1-Y) / (1-Y_hat) )
'''

def L_model_backward(Y_hat, Y, caches):
    '''
    input 중에 
    caches = [cache-0, cache-1, ... ,cache-2,... ,cache-L-1] -> L개
                  ↓
              cache-l = (linear_cache, activation_cache)

    output: grads {
    "dA1" : ...   , "dW1" : ... , "db1" : ... 
    "dA2" : ...   , "dW2" : ... , "db2" : ... 
    .....
    "dAL" : ...   , "dWL" : ... , "dbL" : ... }

    dZ는요? 걔는 dw da db를 계산하기 위한 라면받침일 뿐입니다
    '''
    grads = {}
    L = len(caches)
    m = Y_hat.shape[1]
    Y = Y.reshape(Y_hat.shape)

    # dAL = - ( Y/Y_hat   -   (1-Y) / (1-Y_hat) ) -> y와 y_hat 을 통해 계산가능
    dAL = - (np.divide(Y,Y_hat) - np.divide((1-Y),(1-Y_hat)))

    #최초의 backward
    current_cache = caches[-1]
    dA_prev_temp, dW_temp, db_temp = backward_with_activation(dAL, current_cache, activation = "sigmoid")

    grads["dA"+str(L-1)] = dA_prev_temp 
    grads["dW"+str(L)] = dW_temp
    grads["db"+str(L)] = db_temp

    #L-1번째 LAYER부터 L번째 레이어까지
    for l in reversed(range(L-1)):
        #l = L-2 L-3 ,..., 2, 1, 0
        #l번째에는 relu가 쓰임
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = backward_with_activation(dAL, current_cache, activation = "relu")
        grads["dA"+str(l)] = dA_prev_temp #dA는 dA0 ~ dAL-1 + dAL
        grads["dW"+str(l+1)] = dW_temp #dW1 ~ dWL
        grads["db"+str(l+1)] = db_temp #db1 ~ dbL
    
    return grads

    

In [110]:
def update_params(params, grads, lr = 0.03):
    '''
    W = W - lr * dW    
    b = b - lr * db
    w,b는 params에, dw와 db는 grads에 있음
    params = {W1 : ... , b1 : ... , ~ , WL : ... , BL : ...} 
    grads  = {dA0: ... , dW1: ... , db1: ... , ~ , dAL: ..., dWL : ..., dWb : ...}
    '''
    parameters = copy.deepcopy(params)
    L = len(params) // 2 #float가 아닌 int를 반환하기 위해 //사용

    for l in range(1,L+1):
        # 위 params과 grads를 통해 업데이트를 하기 위해 다음과 같이 활용
        parameters['W' + str(l)] = parameters['W' + str(l)] - lr * grads['dW'+str(l)]
        parameters['b' + str(l)] = parameters['b' + str(l)] - lr * grads['db'+str(l)]
    return parameters